In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/hastie"
results_dir_tanh = "results/hastie"
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]


full_config_path = "hastie_N80_p20"

tanh_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)


In [ ]:
from sklearn.metrics import mean_squared_error
from properscoring import crps_ensemble
import numpy as np
import pandas as pd

# IMPORTANT: this y_test must correspond to the same test set used to make `output_test` in Stan,
# otherwise scores won’t be comparable.
from utils.generate_data import generate_latent_hastie_data
X_train, X_test, y_train, y_test = generate_latent_hastie_data(n=100, p=20)

rows = []
for model_name, model_entry in tanh_fit.items():
    post = model_entry["posterior"]

    # (S, n_test)
    y_samps = post.stan_variable("output_test").squeeze(-1)

    # Optional: limit to first S draws if desired
    # S = min(4000, y_samps.shape[0])
    # y_samps = y_samps[:S]

    # Posterior-mean predictions and RMSE
    y_mean = y_samps.mean(axis=0)                                   # (n_test,)
    rmse_post_mean = float(np.sqrt(mean_squared_error(y_test, y_mean)))

    # Per-draw RMSEs and their mean
    per_draw_rmse = np.sqrt(((y_samps - y_test[None, :])**2).mean(axis=1))  # (S,)
    rmse_draw_mean = float(per_draw_rmse.mean())

    # CRPS across the ensemble (expects shape (n_test, S))
    crps = float(np.mean(crps_ensemble(y_test, y_samps.T)))

    rows.append({
        "Model": model_name,
        "RMSE_posterior_mean": rmse_post_mean,
        "RMSE_mean_over_draws": rmse_draw_mean,
        "CRPS": crps,
        "n_draws": y_samps.shape[0]
    })

results_df = pd.DataFrame(rows).sort_values("RMSE_posterior_mean")
print(results_df)


In [4]:
from sklearn.model_selection import train_test_split
def make_latent_data_sec54(n, p, d=5, r_theta=1.0, sigma_xi=0.0, random_state=42, test_size=0.2):
    """
    Section 5.4 latent model (Hastie–Montanari–Rosset–Tibshirani):
      X = Z W^T + U,   y = Z θ + ξ
      z_i ~ N(0, I_d), u_ij ~ N(0, 1), ξ_i ~ N(0, σ_ξ^2)
    Rows w_j of W satisfy ||w_j|| = 1.               [Fig. 5/6 setup]
    Population mapping to linear model:
      Σ = I_p + W W^T,   β = W (I + W^T W)^{-1} θ.   [eqs. (26)-(27)]
    Returns: X (n×p), y (n,), W (p×d), theta (d,), beta_true (p,), Sigma (p×p)
    """
    rng = np.random.default_rng(random_state)

    # Random W with unit-norm rows (||w_j||=1)
    W = rng.normal(size=(p, d))
    W /= np.linalg.norm(W, axis=1, keepdims=True) + 1e-12  # enforce ||w_j||=1

    # Latent Z, feature noise U, label noise ξ
    Z = rng.normal(size=(n, d))
    U = rng.normal(size=(n, p))
    xi = rng.normal(scale=sigma_xi, size=n)

    # Signal vector θ with ||θ|| = r_theta
    theta = rng.normal(size=d)
    theta *= r_theta / (np.linalg.norm(theta) + 1e-12)

    # Data
    X = Z @ W.T + U
    y = Z @ theta + xi

    # Population quantities for risk
    Sigma = np.eye(p) + W @ W.T
    beta_true = W @ np.linalg.solve(np.eye(d) + W.T @ W, theta)  # β = W (I + W^T W)^(-1) θ
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, Z, y_train, W, theta, beta_true, Sigma

X_train, Z, y_train, W, theta, beta_true, Sigma = make_latent_data_sec54(100, 20)

In [5]:
import numpy as np

def bayes_rmse_floor(W, theta, sigma_xi):
    Ainv = np.linalg.inv(np.eye(W.shape[1]) + W.T @ W)         # (I + W^T W)^{-1}
    mse = float(theta.T @ Ainv @ theta) + sigma_xi**2
    return np.sqrt(mse)



In [ ]:
bayes_rmse_floor(W, theta, sigma_xi=0)

In [7]:
from utils.generate_data import generate_latent_hastie_data
def compute_sparse_rmse_results_hastie(models, all_fits, forward_pass,
                         sparsity=0.0, prune_fn=None):
    results = []
    posterior_means = []
    _, X_test, _, y_test = generate_latent_hastie_data(n=100, p=20)
    for model in models:
        try:
            fit = all_fits[model]['posterior']
            W1_samples = fit.stan_variable("W_1")           # (S, P, H)
            W2_samples = fit.stan_variable("W_L")           # (S, H, O)
            b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
            b2_samples = fit.stan_variable("output_bias")   # (S, O)
        except KeyError:
            print(f"[SKIP] Model or posterior not found:")
            continue

        S = W1_samples.shape[0]
        rmses = np.zeros(S)
        #print(y_test.shape)
        y_hats = np.zeros((S, y_test.shape[0]))

        for i in range(S):
            W1 = W1_samples[i]
            W2 = W2_samples[i]

            # Apply pruning mask if requested
            if prune_fn is not None and sparsity > 0.0:
                masks = prune_fn([W1, W2], sparsity)
                W1 = W1 * masks[0]
                #W2 = W2 * masks[1]

            y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i])
            y_hats[i] = y_hat.squeeze()  # Store the prediction for each sample
            rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test)**2))
            
        posterior_mean = np.mean(y_hats, axis=0)
        posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test.squeeze())**2))

        posterior_means.append({
            'model': model,
            'sparsity': sparsity,
            'posterior_mean_rmse': posterior_mean_rmse
        })

        for i in range(S):
            results.append({
                'model': model,
                'sparsity': sparsity,
                'rmse': rmses[i]
            })

    df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_rmse, df_posterior_rmse


In [8]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

df_rmse_tanh, df_posterior_rmse_tanh = {}, {}

for sparsity in sparsity_levels:

    df_rmse_tanh[sparsity], df_posterior_rmse_tanh[sparsity] = compute_sparse_rmse_results_hastie(
        models = model_names_tanh,
        all_fits = tanh_fit, 
        forward_pass = forward_pass_tanh,
        sparsity=sparsity, 
        prune_fn=local_prune_weights
    )


In [ ]:
# python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---- Anta df_rmse_tanh er definert som i spørsmålet ----
# df_rmse_tanh: Dict[float, DataFrame(model, sparsity, rmse)]

# ---- Slå sammen alle DataFrames og legg til eksplisitt sparsity-kolonne ----
# Hvis de enkelte DF allerede har korrekt 'sparsity', bruker vi den.
# Ellers setter vi sparsity fra dictionary-key for robusthet.
frames = []
for sp, df in df_rmse_tanh.items():
    df = df.copy()
    # Sikre at sparsity-kolonnen matcher key (skriver inn hvis avvik)
    df['sparsity'] = float(sp)
    frames.append(df)

df_all = pd.concat(frames, axis=0, ignore_index=True)

# Rydding: sikre dtypes og sortering
df_all['sparsity'] = df_all['sparsity'].astype(float)
df_all['rmse'] = pd.to_numeric(df_all['rmse'], errors='coerce')
df_all = df_all.dropna(subset=['rmse', 'sparsity', 'model'])

# Valgfritt: sortér modellnavn som kategorisk for konsistent plottrekkefølge
models_order = sorted(df_all['model'].unique())
df_all['model'] = pd.Categorical(df_all['model'], categories=models_order, ordered=True)

# ---- Oppsummeringsstatistikk: mean, std, n, 95% CI ----
summary = (
    df_all.groupby(['model', 'sparsity'])
    .agg(n=('rmse', 'size'),
         mean_rmse=('rmse', 'mean'),
         std_rmse=('rmse', 'std'))
    .reset_index()
)

# Unngå deling på null
summary['sem'] = summary['std_rmse'] / summary['n'].replace(0, np.nan).pow(0.5)
# 95% CI med normaltilnærming: mean ± 1.96 * SEM
summary['ci95'] = 1.96 * summary['sem']
summary['ymin'] = summary['mean_rmse'] - summary['ci95']
summary['ymax'] = summary['mean_rmse'] + summary['ci95']

# ---- Plot-stil ----
sns.set_context('talk')
sns.set_style('whitegrid')

# ---- Figur 1: Linjeplot av mean RMSE vs sparsity, farget per modell, med CI ----
plt.figure(figsize=(10, 6))
# Linjer
sns.lineplot(
    data=summary.sort_values(['model', 'sparsity']),
    x='sparsity', y='mean_rmse', hue='model',
    linewidth=2.5, marker='o', markersize=7
)
# Errorbars (CI)
for _, row in summary.iterrows():
    plt.plot([row['sparsity'], row['sparsity']], [row['ymin'], row['ymax']],
             color=sns.color_palette()[models_order.index(row['model'])], lw=2)

plt.title('RMSE vs sparsity per modell (mean ± 95% CI)')
plt.xlabel('Sparsity')
plt.ylabel('RMSE')
plt.legend(title='Modell', loc='best')
plt.tight_layout()

# ---- Figur 2: Boxplot av RMSE fordelt per sparsity, delt (hue) på modell ----
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df_all,
    x='sparsity', y='rmse', hue='model',
    showfliers=False,  # skjul outliers for ryddigere helhetsinntrykk
    linewidth=1.2
)
sns.stripplot(
    data=df_all.sample(min(len(df_all), 2000), random_state=42),  # vis et utvalg punkter, ikke alt
    x='sparsity', y='rmse', hue='model',
    dodge=True, size=2, alpha=0.25, palette='dark'
)
# Fjern duplisert legend fra stripplot
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[:len(models_order)], labels[:len(models_order)], title='Modell', loc='best')

plt.title('RMSE-fordeling per sparsity og modell (box + punkter)')
plt.xlabel('Sparsity')
plt.ylabel('RMSE')
plt.tight_layout()

plt.show()


## POSTERIOR ANALYSIS

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from utils.generate_data import generate_latent_hastie_data
X_train, _, _, _ = generate_latent_hastie_data(n=100, p=20)

P = 20
H = 16
L = 1
out_nodes = 1

layer_structure = {
    'input_to_hidden': {'name': 'W_1', 'shape': (P, H)},
    'hidden_to_output': {'name': 'W_L', 'shape': (H, out_nodes)}
}


def build_single_draw_weights(fits, layer_structure, draw_idx):
    """Return {model: {'W_1': (P,H), 'W_L': (H,O)}} for ONE draw."""
    out = {}
    for name, fd in fits.items():
        fit = fd["posterior"]
        W1 = fit.stan_variable(layer_structure['input_to_hidden']['name'])[draw_idx]
        WL = fit.stan_variable(layer_structure['hidden_to_output']['name'])[draw_idx]
        WL = WL.reshape(layer_structure['hidden_to_output']['shape'])
        out[name] = {"W_1": W1, "W_L": WL}
    return out

def scale_W1_for_plot(model_means, mode='global'):
    """
    Skalerer alle W_1 til [-1, 1] for rettferdig sammenligning av edge-tykkelser.

    mode:
      - 'global' : én felles skala over alle modeller (mest sammenlignbar)
      - 'per_model': egen skala per modell (uavhengig sammenligning)
      - 'per_node' : skalerer hver kolonne (node) separat til [-1,1]

    Returnerer: scaled_model_means (samme struktur som input), scale_info
    """
    scaled = {}
    if mode == 'global':
        gmax = max(np.abs(m['W_1']).max() for m in model_means.values())
        gmax = max(gmax, 1e-12)
        for name, m in model_means.items():
            W1s = m['W_1'] / gmax
            out = {k: v for k, v in m.items()}
            out['W_1'] = W1s
            scaled[name] = out
        return scaled, {'mode': 'global', 'scale': gmax}

    elif mode == 'per_model':
        for name, m in model_means.items():
            s = max(np.abs(m['W_1']).max(), 1e-12)
            out = {k: v for k, v in m.items()}
            out['W_1'] = m['W_1'] / s
            scaled[name] = out
        return scaled, {'mode': 'per_model'}

    elif mode == 'per_node':
        for name, m in model_means.items():
            W1 = m['W_1'].copy()
            P, H = W1.shape
            for h in range(H):
                colmax = max(np.abs(W1[:, h]).max(), 1e-12)
                W1[:, h] = W1[:, h] / colmax
            out = {k: v for k, v in m.items()}
            out['W_1'] = W1
            scaled[name] = out
        return scaled, {'mode': 'per_node'}

    else:
        raise ValueError("mode must be 'global', 'per_model', or 'per_node'")
#feature_names = list(X_train.columns)
def plot_models_with_activations(model_means, layer_sizes,
                                 activations=None, activation_color_max=None,
                                 ncols=3, figsize_per_plot=(5,4), signed_colors=False, feature_names=None):
    """
    model_means: dict {model_name: {'W_1':(P,H), 'W_L':(H,O), optional 'W_internal':[...]} }
    layer_sizes: f.eks [P, H, O] eller [P, H, H, O] ved internlag
    activations: dict {model_name: (H,)} – aktiveringsfrekvens kun for første skjulte lag
    activation_color_max: global maks for skalering av farger (hvis None brukes 1.0)
    """
    names = list(model_means.keys())
    n_models = len(names)
    nrows = int(np.ceil(n_models / ncols))
    figsize = (figsize_per_plot[0] * ncols, figsize_per_plot[1] * nrows)

    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    if nrows * ncols == 1:
        axes = np.array([axes])
    axes = axes.flatten()

    # Skru av blanke akser
    for ax in axes[n_models:]:
        ax.axis('off')

    for ax, name in zip(axes, names):
        weights = model_means[name]
        G = nx.DiGraph()
        pos, nodes_per_layer, node_colors = {}, [], []

        # Noder med posisjon og farge
        for li, size in enumerate(layer_sizes):
            ids = []
            ycoords = np.linspace(size - 1, 0, size) - (size - 1) / 2
            for i in range(size):
                nid = f"L{li}_{i}"
                G.add_node(nid)
                pos[nid] = (li, ycoords[i])
                ids.append(nid)
                if li == 0 and feature_names is not None:
                    ax.text(pos[nid][0]-0.12, pos[nid][1], feature_names[i],
                            ha='right', va='center', fontsize=8)

                if activations is not None and li == 1:  # kun første skjulte lag
                    #a = activations.get(name, np.zeros(size))
                    a = activations.get(name, np.zeros(size))
                    a = np.asarray(a).ravel()   # <-- flater til 1D array
                    scale = activation_color_max if activation_color_max is not None else 1.0
                    val = float(np.clip(a[i] / max(scale, 1e-12), 0.0, 1.0))
                    color = plt.cm.winter(val)
                else:
                    color = 'lightgray'
                node_colors.append(color)

            nodes_per_layer.append(ids)

        edge_colors, edge_widths = [], []

        def add_edges(W, inn, ut):
            for j, out_n in enumerate(ut):
                for i, in_n in enumerate(inn):
                    w = float(W[i, j])
                    G.add_edge(in_n, out_n, weight=abs(w))
                    edge_colors.append('red' if w >= 0 else 'blue')
                    edge_widths.append(abs(w))

        # input -> hidden(1)
        add_edges(weights['W_1'], nodes_per_layer[0], nodes_per_layer[1])

        # ev. internlag
        if 'W_internal' in weights:
            for l, Win in enumerate(weights['W_internal']):
                add_edges(Win, nodes_per_layer[l+1], nodes_per_layer[l+2])

        # siste hidden -> output
        add_edges(weights['W_L'], nodes_per_layer[-2], nodes_per_layer[-1])

        nx.draw(G, pos, ax=ax,
                node_color=node_colors,
                edge_color=(edge_colors if signed_colors else 'red'),
                width=[G[u][v]['weight'] for u,v in G.edges()],
                with_labels=False, node_size=400, arrows=False)

        ax.set_title(name, fontsize=10)
        ax.axis('off')

    plt.tight_layout()
    return fig

def compute_hidden_activation(fit_dict, x_train, draw_idx):
    fit = fit_dict['posterior']
    W1 = fit.stan_variable('W_1')[draw_idx, :, :]          # (P,H)
    try:
        b1 = fit.stan_variable('hidden_bias')[draw_idx, :] # (H,)
    except Exception:
        b1 = np.zeros(W1.shape[1])
    # tanh i [-1,1]
    a_full = np.tanh(x_train @ W1 + b1)             # (H,)
    a=np.mean(a_full, axis=0)
    return a


In [ ]:
# Velg en observasjon å "lyse opp" nodefargene med
obs_idx = 3
draw_idx = 69 #pick_draw_idx(prior_fits, seed=42)      # one common draw across models
prior_draws = build_single_draw_weights(tanh_fit, layer_structure, draw_idx)

# 1) Beregn aktivasjoner for ALLE modellene
activations = {}
for name, fd in tanh_fit.items():
    a = compute_hidden_activation(fd, X_train, draw_idx)
    activations[name] = np.abs(a)      

# 2) Skaler vekter for plotting (som før)
scaled, _ = scale_W1_for_plot(prior_draws, mode='per_model')

# 3) Kall plottet med aktivasjoner
# Siden tanh ∈ [-1,1] og vi bruker |a|, så sett activation_color_max=1.0
fig = plot_models_with_activations(
    scaled,
    layer_sizes=[P, H, out_nodes],
    activations=activations,
    activation_color_max=1.0,
    ncols=2,
    feature_names = None #feature_names
)
plt.show()

In [13]:
import numpy as np
from itertools import combinations

def mean_abs(arr):  # arr: (S, ...)
    return np.mean(np.abs(np.asarray(arr)), axis=0)

def nid_single_hidden(posterior, W1_name="W_1", WL_name_candidates=("W_L","W_2")):
    """
    posterior: CmdStanMCMC-objekt
    W_1: shape (S, P, H)  (input -> hidden), som du har
    W_L: shape (S, H) eller (S, H, O)  (hidden -> output)
    """
    W1_samps = posterior.stan_variable(W1_name)          # (S, P, H)
    # Finn navn for siste lag
    for nm in WL_name_candidates:
        try:
            WL_samps = posterior.stan_variable(nm)       # (S, H) eller (S, H, O)
            break
        except Exception:
            WL_samps = None
    if WL_samps is None:
        raise ValueError("Fant ikke siste-lag-vekter (prøv å angi riktig navn i WL_name_candidates).")

    # Posterior plug-in: gjennomsnitt av absoluttverdier
    W1_abs = mean_abs(W1_samps)                          # (P, H)
    WL_abs = mean_abs(WL_samps)                          # (H,) eller (H, O)

    # z^(1): aggregert node-innflytelse (sum over outputs hvis flere)
    if WL_abs.ndim == 1:
        z1 = WL_abs                                      # (H,)
    else:
        z1 = WL_abs.sum(axis=1)                          # (H,)

    P, H = W1_abs.shape

    # Main effects: ω({j}) = Σ_i z_i * |W1[j,i]|
    omega_main = (W1_abs * z1[None, :]).sum(axis=1)      # (P,)

    # Pairwise: ω({j,k}) = Σ_i z_i * min(|W1[j,i]|, |W1[k,i]|)
    omega_pair = np.zeros((P, P))
    for j, k in combinations(range(P), 2):
        mins = np.minimum(W1_abs[j, :], W1_abs[k, :])    # (H,)
        omega = np.dot(z1, mins)                         # skalar
        omega_pair[j, k] = omega_pair[k, j] = omega

    return z1, omega_main, omega_pair


In [ ]:
post = tanh_fit['Gaussian tanh']['posterior'] 
z1, omega_main, omega_pair = nid_single_hidden(post) # W_1=(S,P,H), W_L/(W_2)=(S,H[,O]) # Eksempler: # - topp 10 viktigste noder etter z1: 
top_nodes = np.argsort(-z1) # - topp 10 viktigste features (main effects): 
top_feats = np.argsort(-omega_main) # - sterkeste parvise interaksjoner: 
P = omega_pair.shape[0] 
pairs = [(j, k, omega_pair[j, k]) for j in range(P) for k in range(j+1, P)] 
top_pairs = sorted(pairs, key=lambda t: -t[2])[:10]

res = np.array(omega_main/(np.sum(omega_main)))
print(np.round(res, 3))

In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations

def gini(v):
    v = np.asarray(v, float)
    v = np.abs(v)
    if np.all(v == 0): return 0.0
    v = np.sort(v)
    n = v.size
    cum = np.cumsum(v)
    return (n + 1 - 2 * (cum.sum() / cum[-1])) / n

def topk_share(v, k):
    v = np.asarray(v, float)
    tot = v.sum()
    if tot == 0: return 0.0
    idx = np.argsort(-v)[:k]
    return v[idx].sum() / tot

def summarize_main(omega_main):
    tot = omega_main.sum()
    mx  = omega_main.max() if omega_main.size else 0.0
    return {
        "Gini(main)": gini(omega_main),
        "Top1(main)": topk_share(omega_main, 1),
        "Top3(main)": topk_share(omega_main, 3),
        "Top5(main)": topk_share(omega_main, 5),
        "#≥10%max(main)": int((omega_main >= 0.10 * mx).sum()),
        "Total(main)": tot,
    }

def summarize_pairs(omega_pair):
    # ta øvre trekant uten diagonal
    P = omega_pair.shape[0]
    tri = [omega_pair[j, k] for j in range(P) for k in range(j+1, P)]
    tri = np.asarray(tri, float)
    tot = tri.sum()
    mx  = tri.max() if tri.size else 0.0
    return {
        "Gini(pair)": gini(tri),
        "Top5(pair)": topk_share(tri, 5),
        "Top10(pair)": topk_share(tri, 10),
        "#≥10%max(pair)": int((tri >= 0.10 * mx).sum()),
        "Total(pair)": tot,
    }

models = [
    ("Gaussian tanh",                tanh_fit['Gaussian tanh']['posterior']),
    ("Regularized Horseshoe tanh",   tanh_fit['Regularized Horseshoe tanh']['posterior']),
    ("Dirichlet Horseshoe tanh",     tanh_fit['Dirichlet Horseshoe tanh']['posterior']),
    ("Dirichlet Student T tanh",     tanh_fit['Dirichlet Student T tanh']['posterior']),
]

rows = []
for name, post in models:
    z1, omega_main, omega_pair = nid_single_hidden(post)
    m = summarize_main(omega_main)
    p = summarize_pairs(omega_pair)
    rows.append({
        "Model": name,
        **m,
        **p,
        "Median z1": np.median(z1),
    })

df = pd.DataFrame(rows, columns=[
    "Model",
    "Gini(main)", "Top1(main)", "Top3(main)", "Top5(main)", "#≥10%max(main)", "Total(main)",
    "Gini(pair)", "Top5(pair)", "Top10(pair)", "#≥10%max(pair)", "Total(pair)",
    "Median z1",
])

# Kort og ryddig LaTeX
print(df.to_latex(index=False, float_format="%.3f", escape=False))


## TEST SHAPLEY VALUES

In [ ]:
from utils.generate_data import generate_latent_hastie_data
X_train, X_test, y_train, y_test = generate_latent_hastie_data(n=500, p=20)
print(X_train.shape, X_test.shape)

import pandas as pd

# Suppose X_train has shape (n, p)
X_train_pd = pd.DataFrame(X_train, columns=[f"X{i+1}" for i in range(X_train.shape[1])])
X_test_pd = pd.DataFrame(X_test, columns=[f"X{i+1}" for i in range(X_train.shape[1])])


In [17]:
import numpy as np
import pandas as pd

# W: (p×d), theta: (d,)
alignment = W @ theta   # shape (p,), equals cos(angle) since norms = 1

df_alignment = pd.DataFrame({
    "feature": [f"X{j+1}" for j in range(W.shape[0])],
    "alignment": alignment,
    "abs_alignment": np.abs(alignment)
}).sort_values("abs_alignment", ascending=False)


In [ ]:
import torch
import numpy as np
import shap
import pandas as pd
import matplotlib.pyplot as plt
from utils.robust_utils import build_pytorch_model_from_stan_sample

models_to_eval = [
    "Gaussian tanh",
    "Regularized Horseshoe tanh",
    "Dirichlet Horseshoe tanh",
    "Dirichlet Student T tanh"
]

H = 16                       # hidden dim
feature_names = [f"X{i+1}" for i in range(P)]

# SHAP background & evaluation subsets (reuse across models for fairness)
X_bg   = X_train_pd.sample(n=80, random_state=0).to_numpy(float)
X_eval = X_test_pd.sample(n=40, random_state=1).to_numpy(float)

results = {}   # store mean SHAP vectors per model

for model_name in models_to_eval:

    print(f"\n=== Evaluating SHAP for model: {model_name} ===")

    fit = tanh_fit[model_name]['posterior']

    # Build one representative sample from posterior
    model = build_pytorch_model_from_stan_sample(
        fit, sample_idx=69, input_dim=P, hidden_dim=H,
        output_dim=1, task="regression", activation=torch.tanh
    )
    model.eval()

    def predict_numpy(X_np):
        with torch.no_grad():
            X_t = torch.tensor(X_np, dtype=torch.float32)
            y = model(X_t).cpu().numpy()
        return y

    explainer = shap.KernelExplainer(predict_numpy, X_bg)
    shap_vals = explainer.shap_values(X_eval)  # shape = (n_eval, P)

    mean_abs_shap = np.abs(shap_vals).mean(axis=0)  # global importance

    results[model_name] = mean_abs_shap

    print(f"Top features for {model_name}:")
    order = np.argsort(mean_abs_shap)[::-1]
    for j in order[:10]:
        print(f"  {feature_names[j]:16s}  SHAP={mean_abs_shap[j]:.4f}   align={np.abs(alignment[j]):.4f}")


In [ ]:
plt.figure(figsize=(8,6))

for model_name, shap_imp in results.items():
    plt.scatter(np.abs(alignment), shap_imp, label=model_name, alpha=0.7)
    
plt.scatter(np.abs(alignment), np.abs(beta_true), label="β_true vs alignment")
plt.xlabel("True feature-signal alignment")
plt.ylabel("SHAP importance mean")
plt.title("SHAP vs True Latent Alignment")
plt.legend()
plt.grid(True)
plt.show()


In [26]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression

def alignment_metrics(abs_alignment, shap_imp):
    x = abs_alignment.reshape(-1, 1)
    y = shap_imp.reshape(-1)

    # Pearson & Spearman
    pear = pearsonr(abs_alignment, y)[0]
    spear = spearmanr(abs_alignment, y).correlation

    # Linear fit (no intercept or with intercept — try both)
    lr0 = LinearRegression(fit_intercept=False).fit(x, y)
    r2_0 = lr0.score(x, y)
    slope0 = float(lr0.coef_[0])

    lr1 = LinearRegression(fit_intercept=True).fit(x, y)
    r2_1 = lr1.score(x, y)
    slope1 = float(lr1.coef_[0])
    intercept1 = float(lr1.intercept_)

    return {
        "pearson": pear,
        "spearman": spear,
        "R2_no_intercept": r2_0,
        "slope_no_intercept": slope0,
        "R2_with_intercept": r2_1,
        "slope_with_intercept": slope1,
        "intercept": intercept1,
    }

rows = []
for model_name, shap_imp in results.items():
    m = alignment_metrics(np.abs(alignment), shap_imp)
    m["model"] = model_name
    rows.append(m)

summary_df = pd.DataFrame(rows).set_index("model").sort_values("pearson", ascending=False)


In [ ]:
beta_true = W @ np.linalg.inv(np.eye(W.shape[1]) + W.T @ W) @ theta
summary_df.loc["β_true (theory)"] = alignment_metrics(np.abs(alignment), np.abs(beta_true))
print(summary_df.round(3))


In [ ]:
import pandas as pd
corr = pd.DataFrame(X_train_pd, columns=X_train_pd.columns).drop(columns=["X6", "X7", "X8", "X9", "X10", "X11", "X12", "X13", "X14", "X15", "X16", "X17", "X18", "X19", "X20"]).corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.show()


## TESTING ALIGNMENT